In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
import tensorflow        as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip drive/MyDrive/Eye/file_downloaded

Streaming output truncated to the last 5000 lines.
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-8076309-3.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-224974-31.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-3541867-4.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-7599844-1.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-6625388-3.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-8086850-114.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-5811898-40.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-8986660-45.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-1283307-15.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-9540117-28.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-7563760-33.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-5443411-6.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-5768756-12.jpeg  
  inflating: OCT2017/train/DRUSSEND/DRUSEN/DRUSEN-2257047-100.jpeg  
 

In [ ]:
batch_size = 1

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale            = 1./255
)

test_data = test_datagen.flow_from_directory(
    'OCT2017/test',
    target_size = (256,256),
    batch_size  = batch_size ,
    color_mode  = "grayscale",
    class_mode  = "categorical"
)

Found 1000 images belonging to 4 classes.


In [ ]:
# custom activation function
def custom_activation(output):
  logexpsum = tf.keras.backend.sum(tf.keras.backend.exp(output), axis=-1, keepdims=True)
  result    = logexpsum / (logexpsum + 1.0)

  return result

In [ ]:
##DME VS others

def define_discriminator2(in_shape=(256,256,1), n_classes=2):
  in_image = tf.keras.layers.Input(shape=in_shape)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(in_image)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Dropout(0.2)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Dropout(0.2)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Dropout(0.2)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Flatten()(fe)
  fe = tf.keras.layers.Dropout(0.4)(fe)
  fe = tf.keras.layers.Dense(n_classes)(fe)

  c2_out_layer = tf.keras.layers.Activation("softmax")(fe)
  c2_model     = tf.keras.models.Model(in_image, c2_out_layer)
  c2_model.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00005,  beta_1 = 0.5), metrics = ["accuracy"])
  #tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2)

  d2_out_layer = tf.keras.layers.Lambda(custom_activation)(fe)
  d2_model     = tf.keras.models.Model(in_image, d2_out_layer)
  d2_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00005, beta_1 = 0.5))
  return d2_model, c2_model

In [ ]:
## 3 class discriminator

def define_discriminator3(in_shape=(256,256,1), n_classes=3):
  in_image = tf.keras.layers.Input(shape=in_shape)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(in_image)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  #fe = tf.keras.layers.Dropout(0.7)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  #fe = tf.keras.layers.Dropout(0.7)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  #fe = tf.keras.layers.Dropout(0.7)(fe)

  fe = tf.keras.layers.Conv2D(128, (3,3), strides = (1,1), padding = "same")(fe)
  fe = tf.keras.layers.MaxPool2D(pool_size = (2,2) , strides = (2,2))(fe)
  fe = tf.keras.layers.LeakyReLU(alpha=0.2)(fe)

  fe = tf.keras.layers.Flatten()(fe)
  fe = tf.keras.layers.Dropout(0.4)(fe)
  fe = tf.keras.layers.Dense(n_classes)(fe)

  c3_out_layer = tf.keras.layers.Activation("softmax")(fe)
  c3_model     = tf.keras.models.Model(in_image, c3_out_layer)
  c3_model.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002,  beta_1 = 0.5), metrics = ["accuracy"])

  d3_out_layer = tf.keras.layers.Lambda(custom_activation)(fe)
  d3_model     = tf.keras.models.Model(in_image, d3_out_layer)
  d3_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002, beta_1 = 0.5))
  return d3_model, c3_model

In [ ]:
d2_model, c2_model = define_discriminator2()
d3_model, c3_model = define_discriminator3()

In [ ]:
d3_model.load_weights('/content/drive/MyDrive/GAN_weights/saved_model/c_model/w')
c3_model.load_weights('/content/drive/MyDrive/GAN_weights/saved_model/d_model/w')


In [ ]:
c2_model.load_weights('/content/drive/MyDrive/DME_VS_OTHERS/w')

In [ ]:
predicted = []
real = []
for index , (image,label) in enumerate(test_data):
  if index== 1000:
    break
  output = c2_model.predict(image)
  real.append(np.argmax(label))

  if np.argmax(output) == 0:
    predicted.append(1)
  elif np.argmax(output) == 1:
    output = c3_model.predict(image)
    if np.argmax(output) == 0:
        predicted.append(0)
    elif np.argmax(output) == 1:
        predicted.append(2)
    elif np.argmax(output) == 2:
        predicted.append(3)




In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(real, predicted))
from sklearn.metrics import confusion_matrix

0.973


In [ ]:
confusion_matrix(predicted, real)

array([[245,   1,   0,   0],
       [  3, 230,   0,   0],
       [  1,   0, 250,   2],
       [  1,  19,   0, 248]])

In [ ]:
predicted_categories = tf.argmax(y_pred, axis=1)
true=[]
predicted=[]
for index , (image,label) in enumerate(test_data):
  if index== 1000:
    break
  true.append(label)
  predicted.append(c_model.predict(image))

In [ ]:
predicted_categories = tf.argmax(predicted, axis=1)
true_cat= []
for item in true:
  true_cat.append(np.argmax(item))
predicted_cat= []
for item in predicted:
  predicted_cat.append(np.argmax(item))
confusion_matrix(predicted_cat, true_cat)

# Test Data
## CNV
## Drussen
## DME
## Normal

predicted = []
real = []
for image, label in test_data:
    output = c2_model(image)
    real.append(label)
    If output = 0:
      predicted.append(1)
    If output = 1:
      output = c3_model(image)
      if output == 0:
        predicted.append(0)
      elif output == 1:
        predicted.append(2)
      elif output == 2:
        predicted.append(3)

real , predicted compare